In [2]:
import pandas as pd

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [4]:
#df = pd.read_csv(r'parsed_routes_data\all_routes_data.csv') #real dataset
df = pd.read_csv(r'parsed_routes_data\first_bulk_of_routes_data.csv') # small sample dataset for testing
df.rename(columns={'Unnamed: 0': 'route_id', 'Unnamed: 1': 'num_of_waypoint'}, inplace=True)
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
0,100324.gpx,0,"('50.71662', '12.79072')",50.71662,12.79072
1,100324.gpx,1,"('50.71665', '12.79075')",50.71665,12.79075
2,100324.gpx,2,"('50.71654', '12.79175')",50.71654,12.79175
3,100324.gpx,3,"('50.71649', '12.7922')",50.71649,12.79220
4,100324.gpx,4,"('50.71629', '12.79315')",50.71629,12.79315
...,...,...,...,...,...
907386,99777.gpx,352,"('50.61889', '12.36287')",50.61889,12.36287
907387,99777.gpx,353,"('50.6179', '12.3625')",50.61790,12.36250
907388,99777.gpx,354,"('50.61731', '12.36193')",50.61731,12.36193
907389,99777.gpx,355,"('50.61725', '12.36169')",50.61725,12.36169


In [5]:
#converting str values from 'routes_id' (that is going to become an index) into ints
df['route_id'] = df['route_id'].str.split('.').str[0].astype(int)

#converting str values from 'num_of_waypoint' (that is going to become column names) into ints
df['num_of_waypoint'] = df['num_of_waypoint'].astype(int)

In [6]:
#reshaping df from long format to wide format
df_reshaped = df.pivot(index='route_id', columns='num_of_waypoint', values='lat_lgt')
df_reshaped

num_of_waypoint,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,29802,29803,29804,29805,29806,29807,29808,29809,29810,29811,29812,29813,29814,29815,29816,29817,29818,29819,29820,29821,29822,29823,29824,29825,29826,29827,29828,29829,29830,29831,29832,29833,29834,29835,29836,29837,29838,29839,29840,29841,29842,29843,29844,29845,29846,29847,29848,29849,29850,29851
route_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23211,"('50.6483', '12.98132')","('50.64765', '12.98288')","('50.64732', '12.98357')","('50.64698', '12.98378')","('50.64634', '12.98392')","('50.64624', '12.98405')","('50.64585', '12.98509')","('50.64579', '12.98619')","('50.64528', '12.98776')","('50.645', '12.98884')","('50.64476', '12.98998')","('50.64468', '12.99079')","('50.64464', '12.99168')","('50.64448', '12.9925')","('50.64427', '12.99338')","('50.64404', '12.99398')","('50.64355', '12.99434')","('50.64287', '12.99469')","('50.64177', '12.99515')","('50.64119', '12.99597')","('50.64062', '12.9973')","('50.6396', '12.99934')","('50.63905', '13.00082')","('50.63857', '13.00309')","('50.63811', '13.0009')","('50.63719', '13.00279')","('50.63689', '13.00382')","('50.63627', '13.00489')","('50.63546', '13.00588')","('50.63436', '13.00595')","('50.63365', '13.00444')","('50.63049', '13.00215')","('50.62965', '13.0015')","('50.62867', '13.00185')","('50.6276', '13.00275')","('50.62652', '13.00395')","('50.62535', '13.00429')","('50.62404', '13.00438')","('50.62328', '13.00434')","('50.62224', '13.00455')","('50.62115', '13.00528')","('50.62031', '13.00524')","('50.61908', '13.00571')","('50.61847', '13.00577')","('50.61626', '13.00847')","('50.6153', '13.0097')","('50.61505', '13.0106')","('50.6149', '13.0115')","('50.61391', '13.01281')","('50.61358', '13.01348')",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24218,"('50.75626', '12.18732')","('50.75719', '12.18567')","('50.75736', '12.18572')","('50.7586', '12.18112')","('50.75905', '12.17952')","('50.7602', '12.17874')","('50.76099', '12.17905')","('50.76149', '12.18036')","('50.76199', '12.18158')","('50.76229', '12.18197')","('50.76267', '12.18384')","('50.76241', '12.18158')","('50.76433', '12.18109')","('50.76535', '12.18135')","('50.7667', '12.18124')","('50.76773', '12.18193')","('50.76927', '12.18112')","('50.76989', '12.18154')","('50.77101', '12.18215')","('50.77204', '12.18122')","('50.77299', '12.18204')","('50.77346', '12.18086')","('50.77419', '12.17935')","('50.77506', '12.17843')","('50.77616', '12.17742')","('50.77725', '12.17598')","('50.77821', '12.17528')","('50.77835', '12.17395')","('50.77868', '12.1739')","('50.7786', '12.17289')","('50.77957', '12.17287')","('50.78039', '12.17255')","('50.78056', '12.17326')","('50.78193', '12.17268')","('50.78269', '12.17251')","('50.78315', '12.17518')","('50.78398', '12.1774')","('50.78353', '12.17628')","('50.78311', '12.17425')","('50.78233', '12.16864')","('50.78246', '12.16656')","('50.78195', '12.16457')","('50.7813', '12.16249')","('50.78068', '12.16088')","('50.78005', '12.15978')","('50.77911', '12.15869')","('50.77904', '12.15813')","('50.77863', '12.15753')","('50.77959', '12.15747')","('50.78016', '12.15685')",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26298,"('50.58288', '12.7055')","('50.58261', '12.70659')","('50.58238', '12.70661')","('50.5822', '12.70704')","('50.58119', '12.70788')","('50.58059', '12.70828')","('50.58002', '12.70875')","('50.5793', '12.70896')","('50.5785', '12.70887')","('50.57812', '12.70859')","('50.57751', '12.70827')","('50.57677', '12.70838')","('50

In [7]:
#exploring the new shape
df_reshaped.shape # --> (1830, 29852) (smaller size sample)

#since the df now has 29852 columns, I want to verify, if the longest route really has 29852 waypoints or if something went wrong
df['num_of_waypoint'].max()

#finding out which route has that many waypoints, because I want to have a look into the original gps file
df.sort_values(by = 'num_of_waypoint', ascending = False) #--> it's 2825871.gpx (that hypercorrect documentation of a file should be excluded)

#exploring the distribution of the total number of waypoints per route a bit more
routes_grp = df.groupby(['route_id'])
routes_grp['num_of_waypoint'].max().median() #--> median of total waypoints per route is 252 (smaller size sample)
routes_grp['num_of_waypoint'].max().describe() #--> 75% is at 507 total waypoints: by including 1000 columns I will cover most of the routes and exclude outliers

#dropping rows with more than 1000 waypoints from df_reshaped
df_reshaped[1000].isna().value_counts() #verifing not too many routes would be deteled
df_reshaped = df_reshaped[df_reshaped[1000].isna()]

#dropping every column with only NaN values in it (= columns 1000 and higher)
df_reshaped.isnull().sum(axis=0).value_counts() #verifing that it's column 0-999 that stay
df_reshaped = df_reshaped.dropna(axis=1, how='all')
df_reshaped.shape

(1698, 1000)

In [8]:
df_reshaped
df_reshaped.to_csv('cl1_first_bulk_of_routes_data.csv')